In [ ]:
import os
import sys
import wandb
from huggingface_hub import login

In [ ]:
sys.path.append("../")

In [ ]:
wandb.login()

In [ ]:
login()

In [ ]:
!accelerate launch --mixed_precision="fp16" /w/340/vishnouvina/mobilediffusion/lora_distill_training.py \
    --pretrained_model_name_or_path="SG161222/Realistic_Vision_V4.0" \
    --dataset_name="fantasyfish/laion-art" \
    --caption_column="text"\
    --resolution=512 --center_crop --random_flip \
    --num_train_epochs=5 \
    --gradient_accumulation_steps=4 \
    --gradient_checkpointing \
    --max_train_steps=10 \
    --validation_epochs=1 \
    --distill_level="sd_tiny"\
    --prepare_unet="True"\
    --use_peft\
    --output_weight=0.5\
    --feature_weight=0.5\
    --learning_rate=1e-05 \
    --max_grad_norm=1 \
    --lr_scheduler="constant"\
    --lr_warmup_steps=0 \
    --report_to="wandb" \
    --lora_r=4 \
    --lora_alpha=32 \
    --push_to_hub \
    --hub_model_id="sd-laion-art"\
    --output_dir="sd-laion-art"\

--- Logging error ---
Traceback (most recent call last):
  File "/w/340/vishnouvina/miniconda3/envs/mdenv/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/w/340/vishnouvina/miniconda3/envs/mdenv/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/w/340/vishnouvina/miniconda3/envs/mdenv/lib/python3.9/threading.py", line 937, in _bootstrap
    self._bootstrap_inner()
  File "/w/340/vishnouvina/miniconda3/envs/mdenv/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/w/340/vishnouvina/miniconda3/envs/mdenv/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/w/340/vishnouvina/miniconda3/envs/mdenv/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/w/340/vishnouvina/miniconda3/envs/mdenv/lib/python3.9/site-packages/wandb/

In [ ]:
import torch

print(torch.cuda.is_available())

In [ ]:
#srun --partition biggpunodes --nodelist gpunode1 -c 4 --pty --gres=gpu:1 --mem=48G  accelerate launch --mixed_precision="fp16"  lora_distill_training.py --pretrained_model_name_or_path="SG161222/Realistic_Vision_V4.0" --dataset_name="fantasyfish/laion-art" --caption_column=text --resolution=512 --center_crop --random_flip --train_batch_size=1 --num_train_epochs=6 --gradient_accumulation_steps=4 --gradient_checkpointing --max_train_steps=5000 --distill_level=sd_tiny --prepare_unet=True --use_peft --output_weight=0.5 --feature_weight=0.5 --learning_rate=1e-05 --max_grad_norm=1 --lr_scheduler=constant --lr_warmup_steps=0 --validation_prompts= ["A man in a suit"]  --report_to=wandb --lora_r=4 --lora_alpha=32 --push_to_hub --hub_model_id=laion-art --output_dir=sd-laion-art